\# This notebook has the code for inference**



In [1]:
!pip install transformers
!pip install tokenizer
!pip install datasets
!pip install rouge_score
!pip install sentencepiece
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7b30176933eb26ab747a38bb47151b4f2b2ab72368c0221c4a462d1e75391a00
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
from google.colab import drive
import pandas as pd
import textwrap
import json
import os
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import logging
transformers_logger = logging.getLogger("transformers")
logging.getLogger("transformers").setLevel(logging.ERROR)
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from datasets import load_dataset, load_metric
import torch
from rouge import Rouge


# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


**This section contains the code for loading the json file after pdf extraction and the model initialization**

In [4]:
#Load the json file after pdf extraction
dataset_path = "/content/drive/My Drive/Colab Notebooks/CS5242 Project/"

def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Join the paths
data_file_path = os.path.join(dataset_path, 'testimage16.json')

# Load  data
pdf_data = load_data(data_file_path)

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Define the Model
class SummarizationModel:
    def __init__(self, model_name, device):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = BigBirdPegasusForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
        self.config=BigBirdPegasusForConditionalGeneration.from_pretrained(model_name).config

#Instantiate the model

model_name = "google/bigbird-pegasus-large-arxiv"
model_summarizer = SummarizationModel(model_name, device=DEVICE)
model = model_summarizer.model
tokenizer=model_summarizer.tokenizer
#print(modelsummarizer.config)

#

**Model Inference**

In [6]:
#Generate Summary for the content using the loaded model
def generate_summary(content):
        max_length=300
        num_beams=4
        inputs = tokenizer(content, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs.input_ids.to(DEVICE), max_length=max_length, num_beams=num_beams, early_stopping=True)
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary_text

#Pase each sections and subsection to generate summaries from the model
def process_section(section,results):

    # Process the content of the each section
    section_summary_results = {}
    content = section["Text"]
    section_name=section["Section"]
    summary_text = generate_summary(content)
    section_summary_results["Section Name"] = section_name
    section_summary_results["Generated Summary"] = summary_text
    results.append(section_summary_results)
    print("Section Name: ", section_name)
    wrapped_output = textwrap.fill(str(summary_text), width=80)
    print("Generated Summary: ", wrapped_output)
        # Process the subsections if they exist
    if "Subsections" in section:
        for subsection in section["Subsections"]:
            process_section(subsection,results)


# Summarize the section contents and subsection contents
def summarize_pdf(pdf_data, output_file):
    all_results = []
    for section in pdf_data:
        process_section(section,all_results)
    with open(output_file, "w") as json_file:
        json.dump(all_results, json_file, indent=4)

#Write the final summary to the summary jsonfile
output_file = "summary_results_googlebigbird.json"
summarize_pdf(pdf_data, output_file)


Section Name:  1 Introduction
Generated Summary:  we present a vision transformer ( vit ) for large scale image recognition.<n> we
split an image into patches and provide the sequence of linear embeddings of
these patches as an input to a trans- former.<n> image patches are treated the
same way as tokens ( words ) in an nlp application.<n> we train the model on
image classication in supervised fashion.<n> when trained on mid sized datasets
such as imagenet without strong regularization, these mod- els yield modest
accuracies of a few percentage points below resnets of comparable size.<n> this
seemingly discouraging outcome may be expected : transformers lack some of the
inductive biases fine tuning code and pre trained models are available at
https://github.com/ google research/vision_transformer arxiv:v jun published as
a conference paper at iclr inherent to cnns, such as translation equivariance
and locality, and therefore do not generalize well when trained on insufcient
amounts of 